# Getting Started with Amazon S3 Vectors

S3 Vectors introduces **vector buckets**, a new S3 bucket type with native support for storing and querying vector embeddings at scale. You organize your data inside **vector indexes**, which support similarity search via APIs — without provisioning any infrastructure.

Each vector bucket can contain up to **10,000 indexes**, and each index can hold **tens of millions of vectors**.

This notebook walks through:
- Setting up a vector bucket and index
- Generating embeddings using Cohere via Amazon Bedrock
- Storing and querying vectors


## Installing Requirements

In [18]:
%pip install -U boto3 --quiet
print("Note: you may need to restart the kernel after updating packages.")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel after updating packages.


In addition to boto3:
- Review IAM requirements [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors-iam-policies.html)
- Make sure your embedding model is [enabled on Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html).

## Importing Libraries and Setting Environment

In [ ]:
import boto3
import os
import json
import time

# ⚠️ Recommended: use IAM roles with EC2/SageMaker rather than hardcoded credentials
# os.environ["AWS_ACCESS_KEY_ID"] = '...'
# os.environ["AWS_SECRET_ACCESS_KEY"] = '...'

# Set default region
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'

# Define vector bucket and index names
vector_bucket_name = "testings3vector"
vector_bucket_index = "testings3vectorindex"


## Creating the Vector Bucket and Index

In [20]:
# S3 Vectors client
client = boto3.client('s3vectors')


In [ ]:
# Create the S3 Vector Bucket (idempotent)
response = client.create_vector_bucket(vectorBucketName=vector_bucket_name)


In [22]:
# Create the Vector Index
response = client.create_index(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    dataType='float32',
    dimension=1024,
    distanceMetric='cosine'
)


## Generating Embeddings using Cohere via Amazon Bedrock

In [23]:
# Create a Bedrock runtime client
bedrock = boto3.client("bedrock-runtime")

# Example input text
texts = ["The quick brown fox", "jumps over the lazy dog"]

# Prepare the payload for Cohere
body = {
    "texts": texts,
    "input_type": "search_document"
}
payload = json.dumps(body)

# Use Cohere multilingual embedding model
model_id = "cohere.embed-multilingual-v3"

# Invoke Bedrock model
response = bedrock.invoke_model(
    modelId=model_id,
    body=payload,
    contentType="application/json",
    accept="application/json"
)

# Extract embeddings
response_body = json.loads(response["body"].read())
embeddings = response_body["embeddings"]

# Show sample output
for i, emb in enumerate(embeddings):
    print(f"Text: {texts[i]}")
    print(f"Embedding (dim {len(emb)}): {emb[:5]}...\n")


Text: The quick brown fox
Embedding (dim 1024): [0.01802063, 0.03741455, 0.0413208, 0.024765015, -0.03668213]...

Text: jumps over the lazy dog
Embedding (dim 1024): [0.014190674, 0.033599854, 0.048431396, 0.017120361, -0.009674072]...



## Storing Vector Records

In [24]:
# Store first embedding with a unique key
start = time.perf_counter()
response = client.put_vectors(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    vectors=[
        {
            'key': 'vec1',
            'data': {
                'float32': embeddings[0]
            },
            'metadata': {
                'type': 'doc'
            }
        }
    ]
)
print(f"put_vectors took {time.perf_counter() - start:.3f} seconds")


put_vectors took 0.366 seconds


In [25]:
# Store second embedding with a different key
start = time.perf_counter()
response = client.put_vectors(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    vectors=[
        {
            'key': 'vec2',
            'data': {
                'float32': embeddings[1]
            },
            'metadata': {
                'type': 'doc'
            }
        }
    ]
)
print(f"put_vectors took {time.perf_counter() - start:.3f} seconds")


put_vectors took 0.459 seconds


## Querying for Similar Vectors

In [26]:
start = time.perf_counter()
query = client.query_vectors( 
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    queryVector={"float32": embeddings[0]},
    topK=3,
    returnDistance=True,
    returnMetadata=True
)
elapsed = time.perf_counter() - start
print(f"query_vectors took {elapsed:.3f} seconds")
for result in query["vectors"]:
    print(result)


query_vectors took 0.357 seconds
{'key': 'vec1', 'metadata': {'type': 'doc'}, 'distance': 0.00040334463119506836}
{'key': 'vec2', 'metadata': {'type': 'doc'}, 'distance': 0.42035973072052}


## Optional: Query with Metadata Filtering

In [27]:
query_filtered = client.query_vectors(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    queryVector={"float32": embeddings[0]},
    topK=3,
    returnDistance=True,
    returnMetadata=True,
    filter={"type": "doc"}
)
print("Filtered query results:")
for r in query_filtered["vectors"]:
    print(r)


Filtered query results:
{'key': 'vec1', 'metadata': {'type': 'doc'}, 'distance': 0.00040334463119506836}
{'key': 'vec2', 'metadata': {'type': 'doc'}, 'distance': 0.42035973072052}


## Optional: Listing Stored Vector Keys

In [28]:
response = client.list_vectors(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index,
    maxResults=10
)
print("Vector keys stored in the index:")
for key in response["vectors"]:
    print("-", key)


Vector keys stored in the index:
- {'key': 'vec2'}
- {'key': 'vec1'}


## Cleanup

In [29]:
client.delete_index(
    vectorBucketName=vector_bucket_name,
    indexName=vector_bucket_index
)

client.delete_vector_bucket(vectorBucketName=vector_bucket_name)

{'ResponseMetadata': {'RequestId': 'df4a106e-acb1-4b64-9b73-3455bf11c06a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 18 Jul 2025 10:46:29 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amz-request-id': 'df4a106e-acb1-4b64-9b73-3455bf11c06a',
   'access-control-allow-origin': '*',
   'vary': 'origin, access-control-request-method, access-control-request-headers',
   'access-control-expose-headers': '*'},
  'RetryAttempts': 0}}

### Summary

- `put_vectors()` stores individual vectors with a **unique `key`**. Reusing a key will overwrite the previous record.
- `query_vectors()` performs similarity search and can return metadata and distances.
- Use `metadata` fields to tag vectors (e.g., type, source) and filter queries.
- `list_keys()` helps inspect what is stored.

Amazon S3 Vectors is a powerful way to build scalable vector search pipelines **without managing infrastructure**.
